In [ ]:
!pip install wandb

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import  Conv2D,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras import Model
from tqdm import tqdm
import wandb
from wandb.keras import WandbCallback
import numpy as np
import cv2

In [ ]:
wandb.init(project="face-recognition")

## loading & preproccesing

In [15]:
dataset_path = "/content/drive/MyDrive/7-7 dataset"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    zoom_range = 0.1,
    brightness_range =  (0.9, 1.1),
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'validation'
)

Found 1090 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


## modeling

In [16]:
batch_size = 32
epochs = 10

In [17]:
class FaceNet(Model):
  def __init__(self):
    super().__init__()

    self.Conv2D_1 = Conv2D(32, (3,3), activation = 'relu',input_shape=(width, height, 3))
    self.Conv2D_2 = Conv2D(64, (3,3), activation = 'relu')
    self.MaxPooling = MaxPooling2D()
    self.flatten = Flatten()
    self.dense_1 = Dense(128, activation = 'relu')
    self.dense_2 = Dense(14, activation='softmax')
    self.dropout = Dropout(0.5)

  def call(self, x):
    y = self.Conv2D_1(x)
    z = self.MaxPooling(y)
    j = self.Conv2D_2(z)
    k = self.MaxPooling(j)
    m = self.flatten(k)
    n = self.dense_1(m)
    w = self.dropout(n)
    out = self.dense_2(w)

    return out


In [18]:
model = FaceNet()

In [19]:
config=wandb.config
config.learning_rate=0.001

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
loss_function = tf.keras.losses.CategoricalCrossentropy()

In [21]:
train_loss = tf.keras.metrics.MeanAbsoluteError()
test_loss = tf.keras.metrics.MeanAbsoluteError()
train_accuracy = tf.keras.metrics.CategoricalAccuracy()
test_accuracy = tf.keras.metrics.CategoricalAccuracy()

In [22]:
for epoch in range (epochs):
  train_accuracy.reset_states()
  test_accuracy.reset_states()
  train_loss.reset_states()
  test_loss.reset_states()
  print("Epoch: ", epoch)

  #Training------------------------------
  for i ,(images, labels) in enumerate(tqdm(train_data)):
    if len(train_data) <= i:
            break
    with tf.GradientTape() as gTape:
      predictions = model(images)
      loss = loss_function(labels, predictions)

      train_loss(labels, predictions)
      train_accuracy(labels, predictions)

    # derivatives calculation
    gradiants = gTape.gradient(loss, model.trainable_variables)

    #update w
    optimizer.apply_gradients(zip(gradiants, model.trainable_variables))

  #Validation---------------------------
  for i, (images, labels) in enumerate(tqdm(val_data)):
    if len(train_data) <= i:
            break
    predictions = model(images)
    loss = loss_function(labels, predictions)
    test_accuracy(labels, predictions)
    test_loss(labels, predictions)

  print("Train Accuracy: ",train_accuracy.result())
  print("Test Accuracy: ",test_accuracy.result())
  print("Train loss : ", train_loss.result())
  print("Test loss : ", test_loss.result())

  wandb.log({'epochs':  epoch ,
              'Train_loss': np.mean(train_loss.result()),
              'Train_accuracy': float(train_accuracy.result()),
              'val_loss': np.mean(test_loss.result()),
              'val_accuracy':float(test_accuracy.result())})

Epoch:  0


100%|██████████| 35/35 [06:57<00:00, 11.93s/it]
35it [01:56,  3.33s/it]


Train Accuracy:  tf.Tensor(0.16055046, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.24528302, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.13139245, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.12298011, shape=(), dtype=float32)
Epoch:  1


100%|██████████| 35/35 [00:25<00:00,  1.38it/s]
35it [00:20,  1.71it/s]


Train Accuracy:  tf.Tensor(0.4293578, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.36792454, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.10381867, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.09756249, shape=(), dtype=float32)
Epoch:  2


...

Train Accuracy:  tf.Tensor(0.9183486, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.83113205, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.022884715, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.032370552, shape=(), dtype=float32)
Epoch:  8


100%|██████████| 35/35 [00:21<00:00,  1.60it/s]
35it [00:21,  1.63it/s]


Train Accuracy:  tf.Tensor(0.9550459, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.76981133, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.012842991, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.038506065, shape=(), dtype=float32)
Epoch:  9


100%|██████████| 35/35 [00:21<00:00,  1.62it/s]
35it [00:21,  1.59it/s]


Train Accuracy:  tf.Tensor(0.96605504, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8537736, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.011018285, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.028111264, shape=(), dtype=float32)


In [24]:
model.save_weights("FaceNet.h5")